In [18]:
import gradio as gr
import os

import logging

import librosa
import torch

import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
import numpy as np

In [2]:
%cd G:\AI\so-vits-svc_v2\VITS_WebUI\monotonic_align
!python setup.py build_ext --inplace
%cd ..

G:\AI\so-vits-svc_v2\VITS_WebUI\monotonic_align
G:\AI\so-vits-svc_v2\VITS_WebUI


In [4]:
def resize2d(source, target_len):
    source[source<0.001] = np.nan
    target = np.interp(np.arange(0, len(source)*target_len, len(source))/ target_len, np.arange(0, len(source)), source)
    return np.nan_to_num(target)
def convert_wav_22050_to_f0(audio):
    tmp = librosa.pyin(audio,
                fmin=librosa.note_to_hz('C0'),
                fmax=librosa.note_to_hz('C7'),
                frame_length=1780)[0]
    f0 = np.zeros_like(tmp)
    f0[tmp>0] = tmp[tmp>0]
    return f0

In [5]:
def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    print(text_norm.shape)
    return text_norm

In [10]:
CONFIG_PATH = "configs/config.json"
MODEL_PATH = "models/Yuuka/Yuuka.pth"

hps = utils.get_hparams_from_file(CONFIG_PATH)
net_g_ms = SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()



In [8]:
_ = utils.load_checkpoint(MODEL_PATH, net_g_ms, None)

INFO:root:Loaded checkpoint 'models/Yuuka/Yuuka.pth' (iteration 445)


In [11]:
hubert = torch.hub.load("bshall/hubert:main", "hubert_soft")

Using cache found in C:\Users\l4227/.cache\torch\hub\bshall_hubert_main


In [12]:
def vc_fn(input_audio,vc_transform):
    if input_audio is None:
        return "You need to upload an audio", None
    sampling_rate, audio = input_audio
    # print(audio.shape,sampling_rate)
    duration = audio.shape[0] / sampling_rate
    if duration > 30:
        return "Error: Audio is too long", None
    audio = (audio / np.iinfo(audio.dtype).max).astype(np.float32)
    if len(audio.shape) > 1:
        audio = librosa.to_mono(audio.transpose(1, 0))
    if sampling_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sampling_rate, target_sr=16000)

    audio22050 = librosa.resample(audio, orig_sr=16000, target_sr=22050)
    f0 = convert_wav_22050_to_f0(audio22050)

    source = torch.FloatTensor(audio).unsqueeze(0).unsqueeze(0)
    print(source.shape)
    with torch.inference_mode():
        units = hubert.units(source)
        soft = units.squeeze(0).numpy()
        print(sampling_rate)
        f0 = resize2d(f0, len(soft[:, 0])) * vc_transform
        soft[:, 0] = f0 / 10
    sid = torch.LongTensor([0])
    stn_tst = torch.FloatTensor(soft)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g_ms.infer(x_tst, x_tst_lengths,sid=sid, noise_scale=0.1, noise_scale_w=0.1, length_scale=1)[0][
            0, 0].data.float().numpy()

    return "Success", (hps.data.sampling_rate, audio)

In [19]:
app = gr.Blocks()
with app:
    with gr.Tabs():
        with gr.TabItem("Basic"):
            vc_input3 = gr.Audio(label="Input Audio (30s limitation)")
            vc_transform = gr.Number(label="transform",value=1.0)
            vc_submit = gr.Button("Convert", variant="primary")
            vc_output1 = gr.Textbox(label="Output Message")
            vc_output2 = gr.Audio(label="Output Audio")
        vc_submit.click(vc_fn, [ vc_input3,vc_transform], [vc_output1, vc_output2])

    app.launch(debug=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:asyncio:Using selector: SelectSelector
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:7890
DEBUG:urllib3.connectionpool:http://127.0.0.1:7890 "GET http://127.0.0.1:7861/startup-events HTTP/1.1" 200 5
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:7890
DEBUG:urllib3.connectionpool:http://127.0.0.1:7890 "HEAD http://127.0.0.1:7861/ HTTP/1.1" 200 0
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


Traceback (most recent call last):
  File "D:\Anaconda\envs\ML\lib\site-packages\soundfile.py", line 161, in <module>
    import _soundfile_data  # ImportError if this doesn't exist
ModuleNotFoundError: No module named '_soundfile_data'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\ML\lib\site-packages\soundfile.py", line 171, in <module>
    _snd = _ffi.dlopen(_libname)
OSError: cannot load library 'D:\Anaconda\envs\ML\Library\bin\sndfile.dll': error 0x7e

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\routes.py", line 394, in run_predict
    output = await app.get_blocks().process_api(
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\blocks.py", line 1075, in process_api
    result = await self.call_function(
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\blocks.py", line 884, in cal

Keyboard interruption in main thread... closing server.
